# Interactive plot with plotly

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.offline import plot
from plotly.graph_objs import *
import plotly.tools as tls  
import plotly.graph_objs as go
init_notebook_mode()

In [ ]:
import numpy as np  # (*) numpy for math functions and arrays

# (*) Pandas for data manipulation
import pandas as pd 

# Read csv file and define dataframe object (df for dataframe)
df = pd.read_csv('data/gapminderDataFiveYear.txt', sep='\t')

df.head()  # show dataframe header to stdout


In [ ]:
# Choose a year, find other years with df['year'].unique()
the_year = 2007   

# Find indices corresponding to 'the_year'
i_year = (df['year'] == the_year)

# Grab all rows correponding to 'the_year'
df_year = df[i_year] 

In [ ]:
colors = dict(
    Asia='#1f77b4', 
    Europe='#ff7f0e', 
    Africa='#2ca02c',
    Americas='#d62728',
    Oceania='#9467bd'
)
# (!) Set 'size' values to be proportional to rendered area,
#     instead of diameter. This makes the range of bubble sizes smaller
sizemode = 'area'       

# (!) Set a reference for 'size' values (i.e. a population-to-pixel scaling).
#     Here the max bubble area will be on the order of 100 pixels
sizeref = df_year['pop'].max() / 1e2**2

# Define a trace-generating function (returns a Scatter object)
def make_trace(X, continent, sizes, color):  
    return Scatter(
        x=X['gdpPercap'],  # GDP on the x-xaxis
        y=X['lifeExp'],    # life Exp on th y-axis
        name=continent,    # label continent names on hover
        mode='markers',    # (!) point markers only on this plot
        marker= Marker(
            color=color,          # marker color
            size=sizes,           # (!) marker sizes (sizes is a list)
            sizeref=sizeref,      # link sizeref
            sizemode=sizemode,    # link sizemode
            opacity=0.6,          # (!) partly transparent markers
            line=Line(width=0.0)  # remove marker borders
        )
    )

In [ ]:


# Initialize data object 
data = Data()

# Group data frame by continent sub-dataframe (named X), 
#   make one trace object per continent and append to data object
for continent, X in df_year.groupby('continent'):
    
    sizes = X['pop']                            # get population array 
    color = colors[continent]                   # get bubble color
    
    data.append(
        make_trace(X, continent, sizes, color)  # append trace to data object
    )                             



In [ ]:
# Set plot and axis titles
title = "Fig 3.1a: Hans Rosling's Bubble Chart for the year {}".format(the_year)
x_title = "Gross Domestic Product per Capita [in USD of the year 2000]"
y_title = "Life Expentancy [in years]"

# Define a dictionary of axis style options
axis_style = dict(     
    zeroline=False,       # remove thick zero line
    gridcolor='#FFFFFF',  # white grid lines
    ticks='outside',      # draw ticks outside axes 
    ticklen=8,            # tick length
    tickwidth=1.5         #   and width
)

# Make layout object
layout = Layout(
    title=title,             # set plot title
    plot_bgcolor='#EFECEA',  # set plot color to grey
    xaxis=XAxis(
        axis_style,      # add axis style dictionary
        title=x_title,   # x-axis title
    ),
    yaxis=YAxis(
        axis_style,      # add axis style dictionary
        title=y_title,   # y-axis title
    )
)

In [ ]:
fig = go.Figure(data=data, layout=layout)
plot(fig)

In [ ]:
# Update 'xaxis' key, set it to log type and with a power exponent format
fig['layout']['xaxis'].update(
    type='log',
    exponentformat='power',
    showexponent='all'
)

# Update the layout object
fig['layout'].update(
    hovermode='closest',  # (!) hover -> closest data pt
    showlegend=False,     # remove legend (info in hover)
    autosize=False,       # turn off autosize
    width=650,            # plot width
    height=500,           # plot height
)

In [ ]:
# Define a hover-text generating function (returns a list of strings)
def make_text(X):
    return 'Country: %s\
    <br>Life Expectancy: %s years\
    <br>GDP per capita: %s $\
    <br>Population: %s million'\
    % (X['country'], X['lifeExp'], X['gdpPercap'], X['pop']/1e6)     
    
# Again, group data frame by continent sub-dataframe (named X),
#   make one trace object per continent and append to data object
i_trace = 0                                        # init. trace counter

for continent, X in df_year.groupby('continent'):
    text = X.apply(make_text, axis=1).tolist()     # get list of hover texts
    fig['data'][i_trace].update(text=text)         # update trace i
    i_trace += 1                                   # inc. trace counter



In [ ]:
# Update layout with an annotation object in 'annotations' (linked to a list)
fig['layout'].update(annotations=Annotations([
    Annotation(
        text='Data source: GapMinder 2007',  # annotation text
        showarrow=False,                     # remove arrow 
        xref='paper',   # use paper coords
        yref='paper',   #  for both x and y coordinates
        x=0.02,         # x-coord (slightly of plotting area edge)
        y=0.98,         # y-coord (slightly of plotting area edge)
        font=Font(size=14),   # increase font size (default is 12)
        bgcolor='#FFFFFF',    # white background
        borderpad=4           # set border/text space (in pixels)
    )
]))  

# Update title
title = "Fig 3.1b: Hans Rosling's Bubble Chart for the year {}".format(the_year)
fig['layout'].update(title=title)


In [ ]:
# (@) Send to Plotly and show in notebook
plot(fig)    